## 배열 연산으로 조건절 표현하기

##### 출처 - 파이썬 라이브러리를 활용한 데이터 분석(2판), 한빛미디어, 웨스 맥키니

In [2]:
import pandas as pd
import numpy as np

In [3]:
xarr = np.array([1.1, 1.2, 1.3, 1.4, 1.5])
xarr

array([1.1, 1.2, 1.3, 1.4, 1.5])

In [5]:
yarr = np.array([2.1, 2.2, 2.3, 2.4, 2.5])
yarr

array([2.1, 2.2, 2.3, 2.4, 2.5])

In [6]:
cond = np.array([True, False, True, True, False])
cond

array([ True, False,  True,  True, False])

#### 예제1- cond의 값이 True일 때는 xarr의 값을, 아닐 때에는 yarr의 값을 취하고 싶을때

In [8]:
# list comprehension 이용 -> 큰 배열 처리가 느림, 다차원 배열에서 사용X

result = [(x if c else y)
         for x,y,c in zip(xarr,yarr, cond)]
result

[1.1, 2.2, 1.3, 1.4, 2.5]

In [10]:
# np.where 이용

result = np.where(cond, xarr, yarr)
result

array([1.1, 2.2, 1.3, 1.4, 2.5])

#### 예제2- 임의로 생성된 행렬에서 양수는 모두 2로, 음수는 -2로 바꾸기

In [11]:
arr = np.random.randn(4,4)
arr

array([[-0.19474839,  0.25669125, -0.16223973,  1.75426197],
       [ 2.05667989, -0.69609021, -0.64035799,  0.31061397],
       [ 0.57994598, -0.66808815, -0.70723028,  0.75805327],
       [ 0.45659723, -1.48283306, -1.05726963, -0.86253541]])

In [12]:
np.where(arr > 0, 2, -2)

array([[-2,  2, -2,  2],
       [ 2, -2, -2,  2],
       [ 2, -2, -2,  2],
       [ 2, -2, -2, -2]])

In [13]:
# np.where을 이용할 때 스칼라값과 배열을 조합할 수 있다. 예를 들어 arr의 모든 양수를 2로 바꿀 수 있다.


np.where(arr > 0, 2, arr)

array([[-0.19474839,  2.        , -0.16223973,  2.        ],
       [ 2.        , -0.69609021, -0.64035799,  2.        ],
       [ 2.        , -0.66808815, -0.70723028,  2.        ],
       [ 2.        , -1.48283306, -1.05726963, -0.86253541]])

#### 원하던 로직 (+all, any)

#### 1. row의 특정 column들에 -1이 하나라도 있을 때 status가 -1

In [44]:
df = pd.DataFrame(
    data=[[1,1,1], [1,1,-1], [-1,-1,-1]],
    index=['row1', 'row2', 'row3'],
    columns=['col1', 'col2', 'col3'])
df

,col1,col2,col3
row1,1,1,1
row2,1,1,-1
row3,-1,-1,-1


In [46]:
df['status'] = np.where(
    df[['col1', 'col2', 'col3']].eq(-1).any(axis=1, skipna=True), -1, None
)
df

,col1,col2,col3,status
row1,1,1,1,None
row2,1,1,-1,-1
row3,-1,-1,-1,-1


#### 2. row의 특정 column들이 모두 0일 때 status가 0

#### - row의 특정 column들에 1과 -1이 모두 있을 때 status가 2

In [26]:
df = pd.DataFrame(
    data=[[1,1,1], [None,1,-1], [-1,-1,-1]],
    index=['row1', 'row2', 'row3'],
    columns=['col1', 'col2', 'col3'])
df

,col1,col2,col3
row1,1.0,1,1
row2,NaN,1,-1
row3,-1.0,-1,-1


In [28]:
df['status'] = np.where(
    np.logical_and(
    df[['col1', 'col2', 'col3']].isin([-1]).any(axis=1, skipna=True),
   df[['col1', 'col2', 'col3']].isin([1]).any(axis=1, skipna=True),
    ), 2, None
)
df

,col1,col2,col3,status
row1,1.0,1,1,None
row2,NaN,1,-1,2
row3,-1.0,-1,-1,None


#### - row의 특정 column이 모두 None일 때 status는 None

In [40]:
df = pd.DataFrame(
    data=[[None,None,None,'status'], [None,1,-1, 'status'], [-1,-1,-1,'status']],
    index=['row1', 'row2', 'row3'],
    columns=['col1', 'col2', 'col3', 'status'])
df

,col1,col2,col3,status
row1,NaN,NaN,NaN,status
row2,NaN,1.0,-1.0,status
row3,-1.0,-1.0,-1.0,status


In [41]:
df['status'] = np.where(
    df[['col1', 'col2', 'col3']].isnull().all(axis=1, skipna=True), None, df['status']
)
df

,col1,col2,col3,status
row1,NaN,NaN,NaN,None
row2,NaN,1.0,-1.0,status
row3,-1.0,-1.0,-1.0,status
